In [1]:
import pandas as pd
import numpy as np
import matplotlib
# matplotlib.use('Agg')
%matplotlib inline
matplotlib.use('module://ipykernel.pylab.backend_inline')
import matplotlib.pyplot as plt
import math


/tmp/ipykernel_30732/205157429.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# 1558 frequency
'''
1	Daily or almost daily
2	Three or four times a week
3	Once or twice a week
4	One to three times a month
5	Special occasions only
6	Never
-3	Prefer not to answer 
'''

# -1 do not know, -3 prefer not to answer
consumption_fields = {
    "red_wine" : {"week": 1568, "month" : 4407},
    "white_wine" : {"week": 1578, "month" : 4418}, # includes champagne
    "beer_cider": {"week": 1588, "month" : 4429},
    "spirits": {"week": 1598, "month" : 4440}, 
     "fortified": {"week": 1608, "month" : 4451},
}

In [3]:
# https://www.gov.uk/weights-measures-and-packaging-the-law/specified-quantities
density_volume = {
    "red_wine": [0.99, 175, "glass"],
    "white_wine": [0.994, 175, "glass"],
    "beer_cider": [1.011, 568, "pint"],
    "fortified" : [0.987, 60, "glass"], # can be either 50 or 70 ml
    "spirits": [0.935, 30, "measure"] # can be either 25 or 35
    
}

In [4]:
unit_weight = {}
for key, value  in density_volume.items():
    unit_weight[key] = value[0] * value[1]

In [5]:

n_samples = None# limit #samples for debug
data_filename = "/projects/prime/ukbb/17_01_2024_selected.tab"
output_dir = "/projects/prime/ukbb/preprocessed_data_2024/"
out_file = output_dir + "alcohol_2024_maart.csv"

withdrawn_participants_file = '/projects/prime/ukbb/withdraw6244_207_20231205.txt'
# data_filename = '~/PRIME/python_raw_cognitive_tmp_distorted.csv'
withdrawn = pd.read_csv(withdrawn_participants_file, header=None)
withdrawn_col = list(withdrawn.iloc[:, 0])

### find out the list of all columns
data_columns = pd.read_table(data_filename, nrows=1, sep='\t').columns

# make a list of columns needed
# 1558 is a frequency
needed_fields = ["f.eid", "f.21022.0.0", "f.1558.0.0", "f.1558.1.0", "f.1558.2.0", "f.1558.3.0"]
for type_of_alc, ukbb_fields in consumption_fields.items():
    print(type_of_alc)
    print(ukbb_fields["week"])
    print(ukbb_fields["month"])
    current_list = [col for col in data_columns if (col.startswith("f." +str(ukbb_fields["week"])) or col.startswith("f." +str(ukbb_fields["month"])))]
    needed_fields = needed_fields + current_list
    print(current_list)
    

red_wine
1568
4407
['f.1568.0.0', 'f.1568.1.0', 'f.1568.2.0', 'f.1568.3.0', 'f.4407.0.0', 'f.4407.1.0', 'f.4407.2.0', 'f.4407.3.0']
white_wine
1578
4418
['f.1578.0.0', 'f.1578.1.0', 'f.1578.2.0', 'f.1578.3.0', 'f.4418.0.0', 'f.4418.1.0', 'f.4418.2.0', 'f.4418.3.0']
beer_cider
1588
4429
['f.1588.0.0', 'f.1588.1.0', 'f.1588.2.0', 'f.1588.3.0', 'f.4429.0.0', 'f.4429.1.0', 'f.4429.2.0', 'f.4429.3.0']
spirits
1598
4440
['f.1598.0.0', 'f.1598.1.0', 'f.1598.2.0', 'f.1598.3.0', 'f.4440.0.0', 'f.4440.1.0', 'f.4440.2.0', 'f.4440.3.0']
fortified
1608
4451
['f.1608.0.0', 'f.1608.1.0', 'f.1608.2.0', 'f.1608.3.0', 'f.4451.0.0', 'f.4451.1.0', 'f.4451.2.0', 'f.4451.3.0']


In [6]:
print("start reading the table .... ")

df = pd.read_table(data_filename, sep='\t', usecols=needed_fields, dtype=str, nrows=n_samples)

print("finished reading the table .... ")
print("# participants before checking entrance age")
print(len(df))
df = df[~df["f.21022.0.0"].isnull()]
print("# participants after checking entrance age but yet with withdrawn ")
print(len(df))

df = df[~df["f.eid"].isin(withdrawn_col)]


print("# participants without withdrawn ")
print(len(df))

start reading the table .... 
finished reading the table .... 
# participants before checking entrance age
502250
# participants after checking entrance age but yet with withdrawn 
502250
# participants without withdrawn 
502250


In [7]:
map_fields = {}
for i in range(4):
    map_fields[f"f.1558.{i}.0"] = f"frequency.{i}"
    for drink, freq in consumption_fields.items():
        old_fld_w =f"f.{freq['week']}.{i}.0"
        old_fld_m =f"f.{freq['month']}.{i}.0"
        new_fld_w = f"{drink}_week.{i}"
        new_fld_m = f"{drink}_month.{i}"
        map_fields[old_fld_w] = new_fld_w
        map_fields[old_fld_m] = new_fld_m
        
df.rename(columns=map_fields, inplace=True)
print(df.columns)

Index(['f.eid', 'frequency.0', 'frequency.1', 'frequency.2', 'frequency.3',
       'red_wine_week.0', 'red_wine_week.1', 'red_wine_week.2',
       'red_wine_week.3', 'white_wine_week.0', 'white_wine_week.1',
       'white_wine_week.2', 'white_wine_week.3', 'beer_cider_week.0',
       'beer_cider_week.1', 'beer_cider_week.2', 'beer_cider_week.3',
       'spirits_week.0', 'spirits_week.1', 'spirits_week.2', 'spirits_week.3',
       'fortified_week.0', 'fortified_week.1', 'fortified_week.2',
       'fortified_week.3', 'red_wine_month.0', 'red_wine_month.1',
       'red_wine_month.2', 'red_wine_month.3', 'white_wine_month.0',
       'white_wine_month.1', 'white_wine_month.2', 'white_wine_month.3',
       'beer_cider_month.0', 'beer_cider_month.1', 'beer_cider_month.2',
       'beer_cider_month.3', 'spirits_month.0', 'spirits_month.1',
       'spirits_month.2', 'spirits_month.3', 'fortified_month.0',
       'fortified_month.1', 'fortified_month.2', 'fortified_month.3',
       'f.21022.0.0']

In [8]:
assignment = {}
print("weekly")
# weekly
for i in range(4):
    print(i)
    for drink in consumption_fields:
        print(drink)
        week_field = f"{drink}_week.{i}"
        df[week_field] = df[week_field].astype(float)
        df.loc[df[week_field].lt(0), week_field] = np.NaN
        
       
        g_pday_field = f"{drink}_gpd.{i}"
        # glass per week * denisty g/ml * volume ml 
        
        assignment[g_pday_field] = (df[ week_field] * density_volume[drink][0] * density_volume[drink][1])/7.0
        print(assignment[g_pday_field])
        
    
df = df.assign(**assignment)   

weekly
0
red_wine
0           0.0
1           0.0
2           NaN
3           NaN
4           NaN
          ...  
502245    148.5
502246    148.5
502247      NaN
502248      NaN
502249    148.5
Name: red_wine_week.0, Length: 502250, dtype: float64
white_wine
0         74.55
1          0.00
2           NaN
3           NaN
4           NaN
          ...  
502245     0.00
502246     0.00
502247      NaN
502248      NaN
502249     0.00
Name: white_wine_week.0, Length: 502250, dtype: float64
beer_cider
0         164.070857
1           0.000000
2                NaN
3                NaN
4                NaN
             ...    
502245      0.000000
502246      0.000000
502247           NaN
502248           NaN
502249     82.035429
Name: beer_cider_week.0, Length: 502250, dtype: float64
spirits
0         0.000000
1         0.000000
2              NaN
3              NaN
4              NaN
            ...   
502245    0.000000
502246    4.007143
502247         NaN
502248         NaN
502249    0.0

In [9]:
# monthly 
assignment = {}
print("monthly")
for i in range(1):
    print(i)
    frequency_field = f"frequency.{i}"
    for drink in consumption_fields:
        print(drink)
        month_field = f"{drink}_month.{i}"
        df[month_field] = df[month_field].astype(float)
        df.loc[df[month_field].lt(0), month_field] = np.NaN
        
        g_pday_field = f"{drink}_gpd.{i}"
        df.loc[df[g_pday_field].isnull(), g_pday_field] = (df[month_field] * density_volume[drink][0] * density_volume[drink][1])/30.437
        
       # never drink alcohol
        df.loc[df[g_pday_field].isnull() & df[frequency_field].eq(6), g_pday_field] = 0
  

monthly
0
red_wine
white_wine
beer_cider
spirits
fortified


In [10]:
df.to_csv(out_file, sep=',')

In [11]:
print(df["red_wine_gpd.0"])

0           0.000000
1           0.000000
2           0.000000
3           0.000000
4           5.692085
             ...    
502245    148.500000
502246    148.500000
502247           NaN
502248           NaN
502249    148.500000
Name: red_wine_gpd.0, Length: 502250, dtype: float64


In [12]:
print(df["white_wine_gpd.0"])

0         74.550000
1          0.000000
2          0.000000
3         11.430167
4          5.715084
            ...    
502245     0.000000
502246     0.000000
502247          NaN
502248          NaN
502249     0.000000
Name: white_wine_gpd.0, Length: 502250, dtype: float64


In [13]:
print(df["beer_cider_gpd.0"])

0         164.070857
1           0.000000
2           0.000000
3           0.000000
4           0.000000
             ...    
502245      0.000000
502246      0.000000
502247           NaN
502248           NaN
502249     82.035429
Name: beer_cider_gpd.0, Length: 502250, dtype: float64


In [14]:
print(df["fortified_gpd.0"])

0          0.00
1         16.92
2          0.00
3          0.00
4          0.00
          ...  
502245     0.00
502246     0.00
502247      NaN
502248      NaN
502249     0.00
Name: fortified_gpd.0, Length: 502250, dtype: float64


In [15]:
print(df["spirits_gpd.0"])

0         0.000000
1         0.000000
2         0.000000
3         0.000000
4         0.000000
            ...   
502245    0.000000
502246    4.007143
502247         NaN
502248         NaN
502249    0.000000
Name: spirits_gpd.0, Length: 502250, dtype: float64
